In [3]:
# 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv'
# 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv'
# 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv'

# Debemos encontrar el ranking de los estados de USA  por su densidad de población en  2010

# 1.  many-to-one merge nombre + población 
# 2. merge on ``state/region``  de  ``pop``,y ``abbreviation`` de ``abbrevs``.
# ``how='outer'`` para tener todos los datos


In [3]:
# Importar las bibliotecas de Python requeridas.
import pandas as pd
import seaborn as sns
import numpy as np

sns.set_theme(style = 'whitegrid')

In [4]:
# Cargar los datasets desde sus fuentes.
df1 = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv',na_values = '?')
df2 = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv',na_values = '?')
df3 = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv',na_values = '?')

In [19]:
# Obtener el cabecero del DataFrame 1.
df1.head()

,abbreviation,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [58]:
# Obtener la cantidad de registros por columna del DataFrame 1.
df1.count()

abbreviation    2544
ages            2544
year            2544
population      2524
dtype: int64

In [59]:
# Verificar si existen datos vacíos o faltantes en el DataFrame 1.
df1.isna().sum()

abbreviation     0
ages             0
year             0
population      20
dtype: int64

In [60]:
# Verificar si existen datos duplicados en el DataFrame 1.
print(f'Cantidad de datos duplicados en el DataFrame 1: {df1.duplicated().sum()}')

Cantidad de datos duplicados en el DataFrame 1: 0


In [61]:
# Obtener el cabecero del DataFrame 2.
df2.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [62]:
# Obtener la cantidad de registros por columna del DataFrame 2.
df2.count()

state            52
area (sq. mi)    52
dtype: int64

In [63]:
# Verificar si existen datos vacíos o faltantes en el DataFrame 2.
df2.isna().sum()

state            0
area (sq. mi)    0
dtype: int64

In [26]:
# Verificar si existen datos duplicados en el DataFrame 2.
print(f'Cantidad de datos duplicados en el DataFrame 2: {df2.duplicated().sum()}')

Cantidad de datos duplicados en el DataFrame 2: 0


In [84]:
# Obtener el cabecero del DataFrame 3.
df3.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [64]:
# Obtener la cantidad de registros por columna del DataFrame 3.
df3.count()

state           51
abbreviation    51
dtype: int64

In [65]:
# Verificar si existen datos vacíos o faltantes en el DataFrame 3.
df3.isna().sum()

state           0
abbreviation    0
dtype: int64

In [66]:
# Verificar si existen datos duplicados en el DataFrame 3.
print(f'Cantidad de datos duplicados en el DataFrame 3: {df3.duplicated().sum()}')

Cantidad de datos duplicados en el DataFrame 3: 0


In [68]:
# Renombrar la columna 'state/region' como 'abbreviation' en el DataFrame 1.
df1.rename(columns = {'state/region': 'abbreviation'}, inplace = True)

In [69]:
# Realizar el merge de los DataFrame 2 y 3 mediante la columna state, y del DataFrame 1 mediante la columna abbreviation.
merged_df = df2.merge(df3, how = 'outer', on = 'state').merge(df1, how = 'inner', on = 'abbreviation')

merged_df

,state,area (sq. mi),abbreviation,ages,year,population
0,Alabama,52423,AL,under18,2012,1117489.0
1,Alabama,52423,AL,total,2012,4817528.0
2,Alabama,52423,AL,under18,2010,1130966.0
3,Alabama,52423,AL,total,2010,4785570.0
4,Alabama,52423,AL,under18,2011,1125763.0
...,...,...,...,...,...,...
2443,Wyoming,97818,WY,under18,1993,137458.0
2444,Wyoming,97818,WY,total,1991,459260.0
2445,Wyoming,97818,WY,under18,1991,136720.0
2446,Wyoming,97818,WY,under18,1990,136078.0


In [56]:
# Obtener la cantidad de registros por columna del DataFrame resultado.
merged_df.count()

state            2448
area (sq. mi)    2448
abbreviation     2448
ages             2448
year             2448
population       2448
dtype: int64

In [55]:
# Verificar si existen datos vacíos o faltantes en el DataFrame resultado.
merged_df.isna().sum()

state            0
area (sq. mi)    0
abbreviation     0
ages             0
year             0
population       0
dtype: int64

In [53]:
# Verificar si existen datos duplicados en el DataFrame resultado.
print(f'Cantidad de datos duplicados en el DataFrame resultado: {merged_df.duplicated().sum()}')

Cantidad de datos duplicados en el DataFrame resultado: 0
